In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import matplotlib.pyplot as plt
from config import key

ModuleNotFoundError: No module named 'config'

In [3]:
# Create a browser for web scrapping.

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# Use browser to visit the iMDB Top 250 web page.

url = 'https://www.imdb.com/chart/top?sort=ir,desc&mode=simple&page=1'
Browser.visit(url)

AttributeError: 'function' object has no attribute 'visit'

In [ ]:
# Soupify the web page.

html = browser.html
imdb = BeautifulSoup(html)
type(imdb)

In [ ]:
# Find the table list of top 250 movies.

top_movies = imdb.find("table", {"data-caller-name":"chart-top250movie"}).find("tbody").find_all("tr")
len(top_movies)

In [ ]:
# Test scrape to find the Movie ID for the first movie in the Top 250 list.

test = top_movies[0]
movie_id1 = test.find("td", {"class": "watchlistColumn"}).find("div")["data-tconst"]
movie_id1

In [ ]:
# Web scrapping loop to extract movie titles and corresponding movie_id.

top_250_ids = []

for movie in top_movies:
    title = movie.find("td", {"class": "titleColumn"}).find("a").text
    movie_id = movie.find("td", {"class": "watchlistColumn"}).find("div")["data-tconst"]
    
    row = {}
    row["title"] = title
    row["movie_id"] = movie_id
    top_250_ids.append(row)
    
movie_ids_df = pd.DataFrame(top_250_ids)
movie_ids_df

In [ ]:
browser.quit()

In [ ]:
# Using the OMDB API to acquire movie metadata. Initial test with The Shawshank Redemption movie ID.

api_url = f"http://www.omdbapi.com/?apikey={key}&i=tt0111161"
response = requests.get(api_url)
response.json()

In [ ]:
# Another API test to look at getting single attributes of an individual movie.

id_code = "tt0111161"
api_url = f"http://www.omdbapi.com/?apikey={key}&i={id_code}"
response = requests.get(api_url)
movie_title = response.json().get("Title")
year_released = response.json().get("Year")
rating = response.json().get("Rated")
genre = response.json().get("Genre")
imdbRating = response.json().get("imdbRating")
Metascore = response.json().get("Metascore")
release_date = response.json().get("Released")
country = response.json().get("Country")
language = response.json().get("Language")
director = response.json().get("Director")
actors = response.json().get("Actors")
imdbVotes = response.json().get("imdbVotes")
earnings = response.json().get("BoxOffice")
imdbID = response.json().get("imdbID")

print(movie_title)
print(year_released)
print(rating)
print(genre)
print(imdbRating)
print(Metascore)
print(release_date)
print(country)
print(language)
print(director)
print(actors)
print(imdbVotes)
print(earnings)
print(imdbID)

In [ ]:
movie_id_list = movie_ids_df['movie_id'].tolist()
len(movie_id_list)

In [ ]:
# Loop to extract all the metadata attributes associated with all of the top 250 movies, and turn into dataframe. 

rows = []

for x in movie_id_list:
    api_url = f"http://www.omdbapi.com/?apikey={key}&i=" + x
    response = requests.get(api_url)
    data = response.json()
    
    rows.append(data)
    
final_df = pd.DataFrame(rows)
final_df

In [ ]:
pd.set_option('display.max_columns', None)
final_df

In [ ]:
# Remove extraneous columns.

clean_df1 = final_df.drop(columns=['Plot', 'Poster', 'Type', 'DVD', 'Production', 'Website', 'Response'])
clean_df1.head()

In [ ]:
clean_df1 = clean_df1.replace("N/A", 0)
clean_df1.head()

In [ ]:
# Convert data to proper types.

clean_df1['Year'] = clean_df1['Year'].astype(int)
clean_df1['Metascore'] = clean_df1['Metascore'].astype(float)
clean_df1['imdbRating'] = clean_df1['imdbRating'].astype(float)
clean_df1['imdbVotes'] = clean_df1['imdbVotes'].str.replace(",", "")
clean_df1['imdbVotes'] = clean_df1['imdbVotes'].astype(int)
clean_df1['Runtime'] = clean_df1['Runtime'].str.replace(" min", "")
clean_df1['Runtime'] = clean_df1['Runtime'].astype(int)

clean_df1.head()

In [ ]:
# More data conversions.
clean_df1.rename(columns = {'Runtime':'Runtime_mins'}, inplace = True)
clean_df1.head()

In [ ]:
clean_df1.rename(columns = {'Actors':'Top_Billed'}, inplace = True)
clean_df1.head()

In [ ]:
clean_df1['BoxOffice'] = clean_df1['BoxOffice'].str.replace("$", "")
clean_df1['BoxOffice'] = clean_df1['BoxOffice'].str.replace(",", "")
clean_df1['BoxOffice'] = clean_df1['BoxOffice'].astype(float)
clean_df1.head()

In [ ]:
clean_df1.info()

In [ ]:
# Adding imdbRank as new column for the Top 250 IMDB movies.

clean_df1.insert(0, 'imdbRank', range(1, 1 + len(clean_df1)))
clean_df1

In [ ]:
# clean_df1.to_csv("IMDB_Cleaned_1.csv", encoding='utf-8', index=False)

In [ ]:
clean_df1["Genre"] = clean_df1.Genre.apply(lambda x: x.split(",")[0])

In [ ]:
clean_df1["Language"] = clean_df1.Language.apply(lambda x: x.split(",")[0])
clean_df1["Country"] = clean_df1.Country.apply(lambda x: x.split(",")[0])
clean_df1.head()

In [ ]:
clean_df1.Language.value_counts()

In [ ]:
clean_df1 = clean_df1.drop(columns=['Ratings'])
clean_df1.head()

In [ ]:
# clean_df1.to_csv("IMDB_Cleaned_1.csv", encoding='utf-8', index=False)

In [ ]:
clean_df1.info()

In [ ]:
clean_df1 = clean_df1.drop(columns=['Released'])
clean_df1.head()

In [ ]:
# clean_df1.to_csv("IMDB_Cleaned_2.csv", encoding='utf-8', index=False)